In [122]:
import numpy as np
import pandas as pd

In [123]:
GEO = "GEODGHR4"
GEN_015 = "GEN_015"
SDC_015 = "SDC_015"
NDVI = "NDVI"

In [124]:
data = pd.read_csv("C:\\Users\\xia_t\\Desktop\\Projects\\youreka\\dataset\\all_subsetted_group.csv")

# filter mental health and indigenous status
data = data.loc[data[SDC_015] <= 2]
data = data.loc[data[GEN_015] <= 5]

In [125]:
# normalize NDVI into a range of [0, 1]
n = data[NDVI]
data[NDVI] = (n - n.min()) / (n.max() - n.min())

In [126]:
# categorize NDVI into N_BINS intervals
N_BINS = 2
bins = np.linspace(0, 1, N_BINS + 1)

for i in range(N_BINS):
    lower = bins[i]
    upper = bins[i + 1]

    inclusive = "right"
    if i == 0:
        inclusive = "both"

    data.loc[data["NDVI"].between(lower, upper, inclusive), "NDVI_BINS"] = i 
    data.loc[data["GEN_015"].between(lower, upper, inclusive), "GEN_015_BINS"] = i

data["NDVI"] = data["NDVI_BINS"]

In [128]:
# sample data (pass)
grouped = data.groupby(SDC_015)
analysis_data = grouped.apply(lambda x: x.sample(250).reset_index(drop=True))

In [129]:
# build contingency table
from scipy.stats.contingency import crosstab

contingency = crosstab(analysis_data[SDC_015], analysis_data[NDVI], analysis_data[GEN_015])

x, y, z, = contingency.elements
contingency_table = contingency.count

In [130]:
# chi-squared test
from scipy.stats import chi2_contingency

chi2_contingency(contingency_table, correction=True)

Chi2ContingencyResult(statistic=44.758954556978466, pvalue=2.295579050685168e-05, dof=13, expected_freq=array([[[28.024, 45.878, 28.25 ,  9.04 ,  1.808],
        [33.976, 55.622, 34.25 , 10.96 ,  2.192]],

       [[28.024, 45.878, 28.25 ,  9.04 ,  1.808],
        [33.976, 55.622, 34.25 , 10.96 ,  2.192]]]))

In [116]:
# chi-squared post-hoc test
from itertools import product, combinations

SDC_015_values = np.arange(2)
NDVI_values = np.arange(2)

groups = product(SDC_015_values, NDVI_values)
group_comparison_combinations = combinations(groups, 2)

for comparison_combination in group_comparison_combinations:
    t = np.transpose(comparison_combination)
    t = tuple(map(tuple, t))

    comparison_contingency_table = contingency_table[t]
    if comparison_contingency_table.min() < 1:
        continue

    chi2, p, dof, ex = chi2_contingency(comparison_contingency_table, correction=False)
    print(f"Chi2 result for pair {comparison_combination}: {chi2}, p-value: {p}")

Chi2 result for pair ((0, 2), (1, 2)): 4.449523101697013, p-value: 0.3485747639609013
Chi2 result for pair ((0, 2), (1, 3)): 1.3038682864450128, p-value: 0.8607188783971189
Chi2 result for pair ((1, 2), (1, 3)): 0.9406016688625382, p-value: 0.9186649489556648


In [117]:
contingency_table.sum()

278